In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [16]:
df_titles_info = pd.read_csv('./output/US_count_vectorizer_dataset.csv')
df_titles_info.head()

,title,filter_title,description,category_id
0,WE WANT TO TALK ABOUT OUR MARRIAGE,we want to talk about our marriage,shantells channel httpswwwyoutubecomshantellma...,People & Blogs
1,The Trump Presidency: Last Week Tonight with J...,the trump presidency last week tonight with jo...,one year after the presidential election john ...,Entertainment
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman rudy mancuso king bach lele pons,watch my previous video ▶ nnsubscribe ► httpsw...,Comedy
3,Nickelback Lyrics: Real or Fake?,nickelback lyrics real or fake,today we find out if link is a nickelback amat...,Entertainment
4,I Dare You: GOING BALD!?,i dare you going bald,i know its been a while since we did this show...,Entertainment


In [17]:
df_x = df_titles_info['filter_title']
df_y = df_titles_info['category_id']

target_names = list(df_titles_info['category_id'].unique())

In [18]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=117)

In [19]:
count_vectorizer = CountVectorizer()
train_count_vector = count_vectorizer.fit_transform(x_train)
test_count_vector = count_vectorizer.transform(x_test)

tfidf_vectorizer = TfidfTransformer()
x_trained_tfidf_vector = tfidf_vectorizer.fit_transform(train_count_vector)
x_test_tfidf_vector = tfidf_vectorizer.transform(test_count_vector)

In [ ]:
svm_tfidf = SVC().fit(x_trained_tfidf_vector, y_train)
svm_pred_tfidf = svm_tfidf.predict(x_test_tfidf_vector)

print("Accuracy: ", accuracy_score(y_test, svm_pred_tfidf))
print(metrics.classification_report(y_test, svm_pred_tfidf, target_names=target_names))

In [ ]:
svm_count = SVC().fit(train_count_vector, y_train)
svm_pred_count = svm_count.predict(test_count_vector)

print("Accuracy: ", accuracy_score(y_test, svm_pred_count))
print(metrics.classification_report(y_test, svm_pred_count, target_names=target_names))